In [12]:
import pandas as pd
import requests
import time
import os

In [21]:
df_marcas = pd.read_csv("marcas.csv")
df_modelos = pd.read_csv("modelos.csv")

if os.path.exists("anos.csv"):
    
    df_anos = pd.read_csv("anos.csv")

    par_modelo = df_modelos[["code", "brand_code"]]
    par_modelo.columns = ["model_code", "brand_code"]
    par_ano = df_anos[["model_code", "brand_code"]]
    diff = pd.concat([par_modelo, par_ano]).drop_duplicates(keep=False)
else:
    diff = df_modelos[["code", "brand_code"]]
    diff.columns = ["model_code", "brand_code"]

df_anos = pd.DataFrame()

for model_code, brand_code in diff.values:
    try:
        response = requests.get(f"https://fipe.parallelum.com.br/api/v2/cars/brands/{brand_code}/models/{model_code}/years")
        response.raise_for_status()   # opcional: levanta exceção em caso de erro HTTP
        texto = response.text        # aqui está todo o conteúdo da resposta em string
        df_temp = pd.DataFrame([{"model_code": model_code, "brand_code": brand_code, "model_ano":texto}])
        df_anos = pd.concat([df_anos, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Ocorreu um erro ao processar o modelo {model_code} da marca {brand_code}: {e}")
        if os.path.exists("anos.csv"):
            print("O arquivo já existe.")
            df_anos.to_csv("anos.csv", index=False, encoding="utf-8", mode="a", header=False)
        else:
            print("O arquivo não existe, e por isso está sendo criado...")
            df_anos.to_csv("anos.csv", index=False, encoding="utf-8")
        
        print("Arquivo salvo. Encerrando o loop para evitar bloqueio por limite de requisições.")
        break
    time.sleep(0.2)  # Pequeno delay para evitar limite de requisições
if not df_anos.empty:
    df_anos.to_csv("anos.csv", index=False, encoding="utf-8")







Ocorreu um erro ao processar o modelo 5292 da marca 7: 429 Client Error: Too Many Requests for url: https://fipe.parallelum.com.br/api/v2/cars/brands/7/models/5292/years
O arquivo não existe, e por isso está sendo criado...
Arquivo salvo. Encerrando o loop para evitar bloqueio por limite de requisições.


In [ ]:
print(df_anos)

In [ ]:
df_modelos = pd.read_csv("modelos.csv")
df_anos = pd.read_csv("anos.csv")
print(df_anos)

par_modelo = df_modelos[["code", "brand_code"]].values
par_ano = df_anos[["model_code", "brand_code"]].values

for model_code, brand_code in par_modelo:
    if (model_code, brand_code) in par_ano:
        print(model_code, brand_code)


par_ano


# EXTRAIR ANO MODELO

In [ ]:
df_marcas = pd.read_csv("marcas.csv")
df_modelos = pd.read_csv("modelos.csv")
df_anos = pd.read_csv("anos.csv")

par_modelo = df_modelos[["code", "brand_code"]].values
par_ano = df_anos[["code", "brand_code"]].values
df_anos = pd.DataFrame()

for code, brand_code in par_modelo:
    if (code, brand_code) not in par_ano:
        try:
            df_temp = pd.read_json(f"https://fipe.parallelum.com.br/api/v2/cars/brands/{brand_code}/models/{code}/years")
            df_temp["model_code"] = code
            df_anos = pd.concat([df_anos, df_temp])
        except Exception as e:
            print(f"Ocorreu um erro ao processar o modelo {code} da marca {brand_code}: {e}")
            if os.path.exists("anos.csv"):
                print("O arquivo já existe.")
                df_anos.to_csv("anos.csv", index=False, encoding="utf-8", mode="a")
            else:
                print("O arquivo não existe, posso criar.")
                df_anos.to_csv("anos.csv", index=False, encoding="utf-8", mode="w")
            
            print("Arquivo salvo. Encerrando o loop para evitar bloqueio por limite de requisições.")
            break
    time.sleep(0.2)  # Pequeno delay para evitar limite de requisições
df_anos.to_csv("anos.csv", index=False, encoding="utf-8")
df_anos